# ELEC3305 - Real Time Sonar App


In [2]:
# Import functions and libraries
import numpy as np
from scipy import signal
from numpy import *
import matplotlib.pyplot as plt
from rtsonar import rtsonar

Loading BokehJS ...

### Task III: Play with different parameters with the real-time sonar!

The function `rtsonar()` in `rtsonar.py` provides a wrapper to create a real-time sonar plot using user-defined processing functions. Copy and paste your functions genPulseTrain(), genChirpPulse(), crossCorr(), findDelay(), and dist2time() to the code cell below. Scroll down to the bottom and you should be able to run `rtsonar()`!

The detection resolution and sensitivity will depend on the parameters you will choose for the sonar. The longer the pulse is, the worse time resolution you will get, but much stronger matched filtering amplitude. You can buy the time resolution by using pulse compression with chirp pulses at the expense of increasing the bandwidth. 

* Run the sonar with different parameters. You can get interesting targets by moving your laptop, moving yourself next to the computer, using a book as a reflecting surface. It would be easier for you to distinguish the target if you move it back and forth. Play with the pulse lengths, the frequency sweeps, etc. 
* To get the best real-time performance, you should restart the kernel every time you run this lab. 
* Submit 4 interesting set of parameters. Explain why you chose them. 

Parameter sets
1. 3000Hz to 5000Hz, unchanged pulse time and repititions. This was chosen since as per the frequency sweep in part 1, my laptop had quite high magnitude response to these frequencies.

2. 12kHz to 16kHz, 200 sample pulse length. Simply to experiment with relatively higher frequencies, the results of this we're wildly innaccurate

3. 4kHz to 16kHz, 1000 sample pulse length. Chosen to try and get very well defined distance measurements. Another observation is that this made the sonar's distance measurements VERY sensitive to directionality. THat is it's field of view got narrower, allowing more precise measurements.

4. 2kHz to 16kHz, 500 sample pulse length. This was chosen to try and extract the best performance out the sonar's distance measurement due to large BW and short pulse length. It also had the advntage of being (relatively) free of spurious background noise. When holding my test board constant the repeated ghost images of the ibject identified in the previous section will very clearly visible in the plot, even up to seven multiples sometimes! Far more than the three at most which could were identified at the end of part 1.


In [3]:
# Copy and paste your 5 functions here:
# genChirpPulse()
def genChirpPulse(Npulse, f0, f1, fs):
    #     Function generates an analytic function of a chirp pulse
    #     Inputs:
    #             Npulse - pulse length in samples
    #             f0     - starting frequency of chirp
    #             f1     - end frequency of chirp
    #             fs     - sampling frequency
    #     Output:
    #              pulse - chirp pulse
    T = Npulse/fs
    t = np.linspace(0,T,Npulse)
    phi = (((f1-f0)/T)/2)*t**2 + f0*t
    chirp = np.exp(1j*2*np.pi*phi)
    return chirp
    
    
# genPulseTrain()
def genPulseTrain(pulse, Nrep, Nseg):
    # Funtion generates a pulse train from a pulse. 
    #Inputs:
    #    pulse = the pulse generated by genChirpPulse
    #    Nrep  =  number of pulse repetitions
    #    Nseg  =  Length of pulse segment >= length(pulse)        
    seg = np.zeros((Nseg))
    seg[0:pulse.size] = pulse[:]
    return np.tile(seg, Nrep)
  
# crossCorr()
def crossCorr( rcv, pulse_a ):
    # Funtion generates cross-correlation between rcv and pulse_a
    # Inputs:
    #    rcv - received signal
    #    pulse_a - analytic pulse
    # Output:
    #    Xrcv - cross-correlation between rcv and pulse_a
    return signal.fftconvolve(rcv, pulse_a[::-1],mode='full')
    


# findDelay()
def findDelay(Xrcv, Nseg):
    # finds the first pulse
    # Inputs:  
    #         Xrcv - the received matched filtered signal
    #         Nseg - length of a segment
    # Output:
    #          idx - index of the beginning of the first pulse
    
    # the logic here is that when cross correlated the peak is when the signal delay
    Xrcv = Xrcv/max(Xrcv)
    delay = np.argmax(Xrcv > 0.5)
    return delay

def dist2time( dist, temperature=21):
    # Converts distance in cm to time in secs
    # Inputs:
    # dist        - distance to object in cm
    # temperature - in celcius
    # Output:
    # t           - time in seconds between transmitted pulse and echo
    
    v_s = 331.5*(np.sqrt(1+ (temperature/273.15)) )
    dist_metres = dist/100
    
    return v_s**-1 * dist_metres * 2

In [16]:
# Run this for Real-time Sonar
# Change the parameters!
# Your functions will need to be efficient for this to work. 
# If you get nothing try thinking about how to speed up your code.
fs = 48000 # Sampling frequency
f0 = 2000 # Chirp initial frequency
f1 = 16000 # Chirp ending frequency

Npulse = 500 # Length of Chirp Pulse
Nrep = 24 # Number of repetition in a pulse train (determines vertical length of plot )
Nseg = 2048*2 # Number of samples between pulses (determines maximum time-of-arrival)
Nplot = 200 # Number of pixels in plot along horizontal direction (higher resolution is slower)
maxdist = 200 # Maximum distance in cm
temperature = 20 # Temperature     


functions = (genChirpPulse, genPulseTrain, crossCorr, findDelay, dist2time) #join the functions together

stop_flag = rtsonar( f0, f1, fs, Npulse, Nseg, Nrep, Nplot, maxdist, temperature, functions )

In [17]:
# Run this to stop the sonar (it will take a few seconds to stop)
stop_flag.set()